In [0]:
CREATE OR REPLACE TABLE data_lake_turing_.prata.populacao_uf (
  WITH crescimento AS (
    SELECT
        sigla_uf,
        ano,
        populacao,
        LAG(populacao) OVER (PARTITION BY sigla_uf ORDER BY ano) AS populacao_anterior,
        ROUND(
            (populacao - LAG(populacao) OVER (PARTITION BY sigla_uf ORDER BY ano)) 
            / LAG(populacao) OVER (PARTITION BY sigla_uf ORDER BY ano) * 100, 2
        ) AS crescimento_percentual
    FROM data_lake_turing_.bronze.br_ibge_populacao_uf
),
media_uf AS (
    SELECT
        sigla_uf,
        AVG(populacao) AS avg_populacao,
        AVG(crescimento_percentual) AS avg_crescimento
    FROM crescimento
    GROUP BY sigla_uf
)
SELECT
    NTILE(4) OVER (ORDER BY avg_populacao) AS quartil,
    sigla_uf,
    avg_populacao,
    avg_crescimento
FROM media_uf
ORDER BY quartil, avg_populacao
)
